<a href="https://colab.research.google.com/github/jchen8000/DemystifyingLLMs/blob/main/6_Deployment/Chatbot_HuggingFace_Hosted_LLM_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Deployment of LLMs

# 6.11 Chatbot, Example of LLM-Powered Application

### **Simple Chatbot Using Hugging Face Hosted LLM (Without LangChain)**

This notebook demonstrates how to build a minimal chatbot using the **Hugging Face Inference Client** directly, without relying on LangChain or other orchestration frameworks.  
It shows how to authenticate securely, send chat-formatted messages to a hosted large language model, maintain conversation history, and produce responses using a fully remote, cloud‑hosted LLM.  
The example uses the instruction‑tuned **Qwen2.5‑7B‑Instruct** model served through Hugging Face’s hosted inference API.

A **Hugging Face access token** is required to run this notebook.  
If you do not already have one, you can obtain it here:  
🔗 https://huggingface.co/docs/hub/security-tokens  
The token should be stored securely as `HF_TOKEN` in **Colab Secrets** or another protected environment variable.

---

### ⚠️ **Compatibility Note**

This notebook is confirmed working as of **January 2026**, including the currently available `Qwen/Qwen2.5-7B-Instruct` model. However, Hugging Face client libraries, model endpoints, and hosted model availability evolve rapidly, and future updates or model deprecations may require adjustments to this notebook. This example serves as a learning-oriented reference illustrating current best practices, not a permanently stable API contract.

---

In [ ]:
%pip install -q \
  huggingface_hub==0.36.0 \
  transformers==4.57.3

In [2]:
import os
from google.colab import userdata
from huggingface_hub import InferenceClient

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HF_TOKEN")
client = InferenceClient()

In [3]:
MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"

def ask_model(message, history, temperature=0.7, max_tokens=512):
    messages = [{"role": "system", "content": "You are a helpful AI assistant."}]

    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})

    messages.append({"role": "user", "content": message})

    try:
        # We pass the model ID here instead of in the Client constructor
        # This allows the 'auto' provider system to work best
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"System Busy: {str(e)}. Please try again in a moment."

In [4]:
def chatbot():
    print("Chatbot initialized. You can start chatting now (type 'quit' to stop)!\n")
    history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit"]:
            break

        answer = ask_model(user_input, history)
        print(f"\nAI: {answer}\n")
        history.append([user_input, answer])

chatbot()

Chatbot initialized. You can start chatting now (type 'quit' to stop)!

You: Hello, How are you?

AI: Hello! I'm just a computer program, so I don't have feelings or emotions. But I'm here and ready to help you with any questions or information you need. How can I assist you today?

You: What are the top 5 largest cities in Canada?

AI: Certainly! Here are the top 5 largest cities in Canada by population:

1. **Toronto** - Population: Approximately 3 million (city proper)
2. **Montreal** - Population: Approximately 1.7 million (city proper)
3. **Vancouver** - Population: Approximately 680,000 (city proper), but the Greater Vancouver Area has over 2.5 million people
4. **Calgary** - Population: Approximately 1.2 million (city proper)
5. **Ottawa** - Population: Approximately 1 million (city proper)

These figures are based on the city proper populations, but if you include the metropolitan areas, the rankings can change. For example, the Greater Toronto Area has over 6.5 million people,